In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

from networks.novel.lite_vae.blocks.haar import HaarTransform
import torch
from networks.novel.lite_vae.encoder            import LiteVAEEncoder
from networks.novel.lite_vae.decoder            import *  # or SDVAEDecoder
from networks.novel.lite_vae.litevae            import LiteVAE

import torch
import torch.nn as nn
from torchinfo import summary
import numpy as np

from data.image_dataset import *
from torch.utils.data import DataLoader
import ptwt, pywt

Instantiating LiteVAEEncoder with model_version: litevae-m..


In [2]:
train_dataset = Image_Dataset('Dataset/busi/busi_train_test_names.pkl', stage="train", excel = False)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=24,
    pin_memory=True,
    drop_last=True,
    shuffle=True,
)

batch = next(iter(train_dataloader))
img_batch = batch['img']
img_batch.shape


torch.Size([24, 3, 224, 224])

In [7]:
base_model = LiteVAE(LiteVAEEncoder(model_version = 'litevae-s'))

Instantiating LiteVAEEncoder with model_version: litevae-s..


In [13]:
# check device of base_model
print(f"Model is on: {next(base_model.parameters()).device}")

dtype = next(base_model.parameters()).dtype
print(f"Model parameters dtype: {dtype}")

Model is on: cpu
Model parameters dtype: torch.float32


In [8]:
out = base_model(img_batch)

DWT Done!
Feature Extraction Done!
Concatenation Done!
Aggregation Done!


In [9]:
out['latent'].shape, out['latent'].device, out['latent'].dtype

(torch.Size([24, 4, 28, 28]), device(type='cpu'), torch.float32)

In [10]:
summary(base_model, input_size = (2, 3, 224, 224))

DWT Done!
Feature Extraction Done!
Concatenation Done!
Aggregation Done!


Layer (type:depth-idx)                        Output Shape              Param #
LiteVAE                                       [2, 4, 28, 28]            2,445,063
├─LiteVAEEncoder: 1-1                         [2, 8, 28, 28]            --
│    └─LiteVAEUNetBlock: 2-1                  [2, 12, 112, 112]         --
│    │    └─Conv2d: 3-1                       [2, 16, 112, 112]         1,744
│    │    └─ModuleList: 3-2                   --                        79,808
│    │    └─MidBlock2D: 3-3                   [2, 32, 112, 112]         37,248
│    │    └─ModuleList: 3-4                   --                        137,664
│    │    └─Conv2d: 3-5                       [2, 12, 112, 112]         1,740
│    └─Downsample2D: 2-2                      [2, 12, 28, 28]           --
│    │    └─AvgPool2d: 3-6                    [2, 12, 28, 28]           --
│    └─LiteVAEUNetBlock: 2-3                  [2, 12, 56, 56]           --
│    │    └─Conv2d: 3-7                       [2, 16, 56, 56]        